## Pytorch

In [ ]:
%%time
import pandas as pd            #pytorch.MLP.Быстрый.ТОП прогноз.Без сезонности(Лутше на краткий срок)
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Загрузка данных из Excel-файла с корректным форматом даты
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/Прогноз SKU/SKU for prophet.xlsx')

data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True, errors='coerce')

# Проверка на наличие некорректных дат и их удаление
invalid_dates = data[data['Дата'].isna()]
if not invalid_dates.empty:
    print(f"Некорректные даты: {invalid_dates}")
    data = data.dropna(subset=['Дата'])

# Переименование столбцов
data = data.rename(columns={'Дата': 'date', 'Код товара': 'SKU', 'Продажи, кг': 'sales'})

# Преобразование данных о продажах в числовой формат
data['sales'] = data['sales'].astype(float)

# Функция для создания лагов и скользящих средних
def create_features(df, lag=6, window=6, is_future=False):
    df = df.copy()
    for i in range(1, lag + 1):
        df[f'lag_{i}'] = df['sales'].shift(i) if not is_future else np.nan
    df['rolling_mean'] = df['sales'].rolling(window=window).mean() if not is_future else np.nan
    df['rolling_std'] = df['sales'].rolling(window=window).std() if not is_future else np.nan
    df['month'] = df['date'].dt.month
    df['dayofweek'] = df['date'].dt.dayofweek
    df['dayofmonth'] = df['date'].dt.day
    df['is_weekend'] = df['dayofweek'].apply(lambda x: 1 if x >= 5 else 0)  # Признак выходных дней
    return df

# Группировка данных по товару
grouped_data = data.groupby('SKU')

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Определение нейронной сети
class SalesPredictor(nn.Module):
    def __init__(self, input_dim):
        super(SalesPredictor, self).__init__()
        self.hidden1 = nn.Linear(input_dim, 100)
        self.hidden2 = nn.Linear(100, 50)
        self.output = nn.Linear(50, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.output(x)
        return x

# Прогнозирование продаж для каждого товара
for sku, sku_data in grouped_data:
    sku_data = create_features(sku_data)
    sku_data = sku_data.dropna()

    if len(sku_data) < 2:  # Пропускаем товары с недостаточным количеством данных
        continue

    # Разделение данных на обучающую и тестовую выборки
    X = sku_data.drop(columns=['date', 'sales', 'SKU'])
    y = sku_data['sales']
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

    # Создание и обучение модели PyTorch
    model = SalesPredictor(input_dim=X_train.shape[1])
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    epochs = 250  # Увеличение числа эпох для лучшего обучения
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
    
    model.eval()
    
    # Прогнозирование на 90 дней вперед
    last_date = sku_data['date'].max()
    future_dates = [last_date + pd.Timedelta(days=i) for i in range(1, 460)]
    future_data = pd.DataFrame({'date': future_dates})
    future_data = create_features(future_data, is_future=True)
    
    # Заполнение лагов и скользящих средних для future_data
    for i in range(1, 7):
        future_data[f'lag_{i}'] = y.iloc[-i] if i <= len(y) else y.iloc[-1]
    future_data['rolling_mean'] = y.rolling(window=6).mean().iloc[-1]
    future_data['rolling_std'] = y.rolling(window=6).std().iloc[-1]
    
    future_data = future_data.drop(columns=['date'])
    future_data = scaler.transform(future_data)
    future_data = torch.tensor(future_data, dtype=torch.float32)

    forecast = model(future_data).detach().numpy()
    forecast_df = pd.DataFrame({
        'date': future_dates,
        'yhat': forecast.flatten(),
        'SKU': sku
    })

    # Замена отрицательных значений прогноза на 0
    forecast_df['yhat'] = forecast_df['yhat'].where(forecast_df['yhat'] > 0, 0)

    # Добавление прогноза для текущего товара к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast_df])

# Сохранение результатов в новом Excel-файле
all_forecasts.to_excel('прогноз_продаж_по_SKU_MLP.xlsx', index=False)

#  PROPHET

In [ ]:
%%time  
                                       # прогноз МОТИВАЦИЯ МАГАЗИНОВ
import pandas as pd
from prophet import Prophet

# Загрузка данных из Excel-файла с корректным форматом даты
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/Прогноз SKU/Продажи для мотивации маг.xlsx')

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True)

# Подготовка данных для применения модели Prophet
data = data.rename(columns={'Дата': 'ds', 'Магазин': 'store', 'Класс2': 'class', 'Продажи,руб': 'y'})
data['y'] = data['y'].astype(float)

# Группировка данных по магазину и Класс2
grouped_data = data.groupby(['store', 'class'])

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Прогнозирование продаж для каждой комбинации магазина и Класс2
for (store, class2), group in grouped_data:
    # Проверка на наличие пропущенных значений в столбце 'y'
    if group['y'].notnull().sum() < 2:
        continue  # Пропускаем прогноз для групп с менее чем 2 непропущенными значениями 'y'

    # Создание и обучение модели Prophet для каждой комбинации магазина и Класс2
    model = Prophet()
    model.fit(group[['ds', 'y']])

    # Создание DataFrame для прогноза на 30 дней вперед
    future = model.make_future_dataframe(periods=40)
    forecast = model.predict(future)
    forecast.loc[forecast['yhat'] < 0, 'yhat'] = 0
    forecast['store'] = store  # Добавление информации о магазине
    forecast['class'] = class2  # Добавление информации о классе

    # Добавление прогноза для текущей комбинации магазина и Класс2 к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast])

# Сохранение результатов в новом Excel-файле
selected_columns = ['ds', 'store', 'class', 'yhat', 'yhat_lower', 'yhat_upper']
all_forecasts[selected_columns].to_excel('прогноз_продаж_мотивация_магазинов.xlsx', index=False)


In [ ]:
%%time                   
import pandas as pd         # прогноз по SKU (Более подходящий) АГРОКОМПЛЕКС и ТД....                  
from prophet import Prophet

# Загрузка данных из Excel-файла с корректным форматом даты
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/Прогноз SKU/SKU for prophet.xlsx')

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True)

# Подготовка данных для применения модели Prophet
data = data.rename(columns={'Дата': 'ds', 'Код товара': 'SKU', 'Продажи, кг': 'y'})
data['y'] = data['y'].astype(float)

# Группировка данных по товару
grouped_data = data.groupby('SKU')

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Прогнозирование продаж для каждого товара
for sku, sku_data in grouped_data:
    # Проверка на наличие пропущенных значений в столбце 'y'
    if sku_data['y'].notnull().sum() < 2:
        continue  # Пропускаем прогноз для товаров с меньше чем 2 непропущенными значениями 'y'

    # Создание и обучение модели Prophet для каждого товара
    model = Prophet()
    model.fit(sku_data[['ds', 'y']])

    # Создание DataFrame для прогноза на 60 дней вперед
    future = model.make_future_dataframe(periods=460)
    forecast = model.predict(future)
    forecast['SKU'] = sku  # Добавление информации о товаре

    # Замена отрицательных значений прогноза на 0
    forecast['yhat'] = forecast['yhat'].where(forecast['yhat'] > 0, 0)

    # Добавление прогноза для текущего товара к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast])

# Сохранение результатов в новом Excel-файле
selected_columns = ['ds', 'SKU', 'yhat', 'yhat_upper']
all_forecasts[selected_columns].to_excel('прогноз_продаж_по_SKU.xlsx', index=False)



# CatBoost

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Загрузка данных из Excel-файла с корректным форматом даты
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/Прогноз SKU/SKU for prophet.xlsx')

data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True, errors='coerce')

# Проверка на наличие некорректных дат и их удаление
invalid_dates = data[data['Дата'].isna()]
if not invalid_dates.empty:
    print(f"Некорректные даты: {invalid_dates}")
    data = data.dropna(subset=['Дата'])

# Переименование столбцов
data = data.rename(columns={'Дата': 'date', 'Код товара': 'SKU', 'Продажи, кг': 'sales'})

# Преобразование данных о продажах в числовой формат
data['sales'] = data['sales'].astype(float)

# Функция для создания лагов и скользящих средних
def create_features(df, lag=6, window=6, is_future=False):
    df = df.copy()
    for i in range(1, lag + 1):
        df[f'lag_{i}'] = df['sales'].shift(i) if not is_future else np.nan
    df['rolling_mean'] = df['sales'].rolling(window=window).mean() if not is_future else np.nan
    df['rolling_std'] = df['sales'].rolling(window=window).std() if not is_future else np.nan
    df['month'] = df['date'].dt.month
    df['dayofweek'] = df['date'].dt.dayofweek
    df['dayofmonth'] = df['date'].dt.day
    df['is_weekend'] = df['dayofweek'].apply(lambda x: 1 if x >= 5 else 0)  # Признак выходных дней
    return df

# Группировка данных по товару
grouped_data = data.groupby('SKU')

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Прогнозирование продаж для каждого товара
for sku, sku_data in grouped_data:
    sku_data = create_features(sku_data)
    sku_data = sku_data.dropna()

    if len(sku_data) < 2:  # Пропускаем товары с недостаточным количеством данных
        continue

    # Разделение данных на обучающую и тестовую выборки
    X = sku_data.drop(columns=['date', 'sales', 'SKU'])
    y = sku_data['sales']

    # Проверка на одинаковые значения в целевой переменной
    if y.nunique() <= 1:
        print(f"Все значения в целевой переменной для товара {sku} одинаковы. Пропускаем...")
        continue

    # Стандартизация данных
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Обучение модели CatBoost
    model = CatBoostRegressor(iterations=1000, learning_rate=0.01, depth=6, silent=True)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=False)
    
    # Прогнозирование на 500 дней вперед
    last_date = sku_data['date'].max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=460)
    future_data = pd.DataFrame({'date': future_dates})
    future_data = create_features(future_data, is_future=True)
    
    # Заполнение лагов и скользящих средних для future_data
    for i in range(1, 7):
        future_data[f'lag_{i}'] = y.iloc[-i] if i <= len(y) else y.iloc[-1]
    future_data['rolling_mean'] = y.rolling(window=6).mean().iloc[-1]
    future_data['rolling_std'] = y.rolling(window=6).std().iloc[-1]
    
    future_data = future_data.drop(columns=['date'])
    future_data = scaler.transform(future_data)
    
    # Прогнозирование
    forecast = model.predict(future_data)
    forecast_df = pd.DataFrame({
        'date': future_dates,
        'yhat': forecast,
        'SKU': sku
    })

    # Замена отрицательных значений прогноза на 0
    forecast_df['yhat'] = forecast_df['yhat'].where(forecast_df['yhat'] > 0, 0)

    # Добавление прогноза для текущего товара к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast_df])

# Сохранение результатов в новом Excel-файле
all_forecasts.to_excel('прогноз_продаж_по_SKU_CatBoost.xlsx', index=False)


# ГИБРИДНЫЕ МОДЕЛИ

In [ ]:
import pandas as pd                                            #ТОП гибридный прогноз на любой срок.
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загрузка данных из Excel-файла с корректным форматом даты
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/Прогноз SKU/SKU for prophet.xlsx')

data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True, errors='coerce')

# Проверка на наличие некорректных дат и их удаление
invalid_dates = data[data['Дата'].isna()]
if not invalid_dates.empty:
    print(f"Некорректные даты: {invalid_dates}")
    data = data.dropna(subset=['Дата'])

# Переименование столбцов
data = data.rename(columns={'Дата': 'date', 'Код товара': 'SKU', 'Продажи, кг': 'sales'})

# Преобразование данных о продажах в числовой формат
data['sales'] = data['sales'].astype(float)

# Функция для создания лагов и скользящих средних
def create_features(df, lag=6, window=6, is_future=False):
    df = df.copy()
    for i in range(1, lag + 1):
        df[f'lag_{i}'] = df['sales'].shift(i) if not is_future else np.nan
    df['rolling_mean'] = df['sales'].rolling(window=window).mean() if not is_future else np.nan
    df['rolling_std'] = df['sales'].rolling(window=window).std() if not is_future else np.nan
    df['month'] = df['date'].dt.month
    df['dayofweek'] = df['date'].dt.dayofweek
    df['dayofmonth'] = df['date'].dt.day
    df['is_weekend'] = df['dayofweek'].apply(lambda x: 1 if x >= 5 else 0)  # Признак выходных дней
    return df

# Группировка данных по товару
grouped_data = data.groupby('SKU')

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Прогнозирование продаж для каждого товара
for sku, sku_data in grouped_data:
    sku_data = create_features(sku_data)
    sku_data = sku_data.dropna()

    if len(sku_data) < 2:  # Пропускаем товары с недостаточным количеством данных
        continue

    # Разделение данных на обучающую и тестовую выборки
    X = sku_data.drop(columns=['date', 'sales', 'SKU'])
    y = sku_data['sales']

    # Проверка на одинаковые значения в целевой переменной
    if y.nunique() <= 1 or len(y) < 10:  # Условие: мало данных или все значения одинаковы
        print(f"Данных для товара {sku} слишком мало или они слишком однородны. Используем среднее значение для прогноза.")
        mean_value = y.mean()
        forecast = [mean_value] * 460  # Прогнозируем среднее значение на 460 дней вперед
        future_dates = pd.date_range(start=sku_data['date'].max() + pd.Timedelta(days=1), periods=460)                # Период епт
        forecast_df = pd.DataFrame({
            'date': future_dates,
            'yhat': forecast,
            'SKU': sku
        })
    else:
        # Стандартизация данных
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        # Обучение модели CatBoost
        model = CatBoostRegressor(iterations=1000, learning_rate=0.01, depth=6, silent=True)
        model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=False)

        # Прогнозирование на 460 дней вперед
        last_date = sku_data['date'].max()
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=460)
        future_data = pd.DataFrame({'date': future_dates})
        future_data = create_features(future_data, is_future=True)

        # Заполнение лагов и скользящих средних для future_data
        for i in range(1, 7):
            future_data[f'lag_{i}'] = y.iloc[-i] if i <= len(y) else y.iloc[-1]
        future_data['rolling_mean'] = y.rolling(window=6).mean().iloc[-1]
        future_data['rolling_std'] = y.rolling(window=6).std().iloc[-1]

        future_data = future_data.drop(columns=['date'])
        future_data = scaler.transform(future_data)

        # Прогнозирование
        forecast = model.predict(future_data)
        forecast_df = pd.DataFrame({
            'date': future_dates,
            'yhat': forecast,
            'SKU': sku
        })

    # Замена отрицательных значений прогноза на 0
    forecast_df['yhat'] = forecast_df['yhat'].where(forecast_df['yhat'] > 0, 0)

    # Сохранение фактических значений для расчета метрик
    if len(y_test) > 0:
        # Расчет метрик
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)  # Коэффициент детерминации

        print(f"SKU: {sku} -  RMSE: {rmse:.2f},  R²: {r2:.2f}")

    # Добавление прогноза для текущего товара к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast_df])

# Сохранение результатов в новом Excel-файле
all_forecasts.to_excel('прогноз_продаж_по_SKU_CatBoost+MLP.xlsx', index=False)


# Стэкинг

In [ ]:
import pandas as pd                                   # ТОП прогноз с сезонностью,но очень медленный(для малоко кол-ва SKU)
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from prophet import Prophet
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загрузка данных из Excel-файла
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/Прогноз SKU/SKU for prophet.xlsx')

data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True, errors='coerce')

# Проверка на наличие некорректных дат и их удаление
invalid_dates = data[data['Дата'].isna()]
if not invalid_dates.empty:
    print(f"Некорректные даты: {invalid_dates}")
    data = data.dropna(subset=['Дата'])

# Переименование столбцов
data = data.rename(columns={'Дата': 'date', 'Код товара': 'SKU', 'Продажи, кг': 'sales'})

# Преобразование данных о продажах в числовой формат
data['sales'] = data['sales'].astype(float)

# Функция для создания лагов и скользящих средних
def create_features(df, lag=6, window=6, is_future=False):
    df = df.copy()
    for i in range(1, lag + 1):
        df[f'lag_{i}'] = df['sales'].shift(i) if not is_future else np.nan
    df['rolling_mean'] = df['sales'].rolling(window=window).mean() if not is_future else np.nan
    df['rolling_std'] = df['sales'].rolling(window=window).std() if not is_future else np.nan
    df['month'] = df['date'].dt.month
    df['dayofweek'] = df['date'].dt.dayofweek
    df['dayofmonth'] = df['date'].dt.day
    df['is_weekend'] = df['dayofweek'].apply(lambda x: 1 if x >= 5 else 0)  # Признак выходных дней
    return df

# Группировка данных по товару
grouped_data = data.groupby('SKU')

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Прогнозирование продаж для каждого товара
for sku, sku_data in grouped_data:
    sku_data = create_features(sku_data)
    sku_data = sku_data.dropna()

    if len(sku_data) < 2:  # Пропускаем товары с недостаточным количеством данных
        continue

    # Разделение данных на обучающую и тестовую выборки
    X = sku_data.drop(columns=['date', 'sales', 'SKU'])
    y = sku_data['sales']

    # Проверка на одинаковые значения в целевой переменной
    if y.nunique() <= 1:
        print(f"Все значения в целевой переменной для товара {sku} одинаковы. Пропускаем...")
        continue

    # Стандартизация данных
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Добавление модели Prophet для учета сезонности
    def train_prophet_model(sku_data):
        # Подготовка данных для Prophet
        prophet_data = sku_data[['date', 'sales']].rename(columns={'date': 'ds', 'sales': 'y'})
        model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
        model.fit(prophet_data)

        # Прогнозирование на основе Prophet
        future_dates = model.make_future_dataframe(periods=460)  # Прогноз на 460 дней
        forecast = model.predict(future_dates)
        return forecast['yhat'].values[-460:]

    # Модель Prophet для сезонности
    prophet_forecast = train_prophet_model(sku_data)

    # Базовые модели
    base_models = [
        ('catboost', CatBoostRegressor(iterations=1000, learning_rate=0.01, depth=6, silent=True)),
        ('rf', RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)),
        ('lr', LinearRegression())
    ]

    # Мета-модель (например, линейная регрессия)
    meta_model = LinearRegression()

    # Стэкинг-регрессор
    stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

    # Обучение стэкинг-модели
    stacking_model.fit(X_train, y_train)

    # Прогнозирование на 460 дней вперед
    last_date = sku_data['date'].max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=460)                  # Период прогноза
    future_data = pd.DataFrame({'date': future_dates})
    future_data = create_features(future_data, is_future=True)

    # Заполнение лагов и скользящих средних для future_data
    for i in range(1, 7):
        future_data[f'lag_{i}'] = y.iloc[-i] if i <= len(y) else y.iloc[-1]
    future_data['rolling_mean'] = y.rolling(window=6).mean().iloc[-1]
    future_data['rolling_std'] = y.rolling(window=6).std().iloc[-1]

    future_data = future_data.drop(columns=['date'])
    future_data = scaler.transform(future_data)

    # Прогнозирование на основе стэкинга
    stacking_forecast = stacking_model.predict(future_data)

    # Усреднение прогнозов Prophet и стэкинг-модели
    final_forecast = (stacking_forecast + prophet_forecast) / 2

    # Создание DataFrame для прогноза
    forecast_df = pd.DataFrame({
        'date': future_dates,
        'yhat': final_forecast,
        'SKU': sku
    })

    # Замена отрицательных значений прогноза на 0
    forecast_df['yhat'] = forecast_df['yhat'].where(forecast_df['yhat'] > 0, 0)

    # Сохранение фактических значений для расчета метрик
    if len(y_test) > 0:
        # Расчет метрик
        y_pred = stacking_model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)

        print(f"SKU: {sku} -  MSE: {mse:.2f},  R²: {r2:.2f}")

    # Добавление прогноза для текущего товара к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast_df])

# Сохранение результатов в новом Excel-файле
all_forecasts.to_excel('прогноз_продаж_по_SKU_Stacking_with_Prophet.xlsx', index=False)


## Neural Prophet

In [ ]:
import os
import pandas as pd
import logging
from neuralprophet import NeuralProphet

# Отключаем автоматическое создание lightning_logs
os.environ['PL_DISABLE_LOGGER'] = '1'

# Отключение логов NeuralProphet
logging.getLogger('neuralprophet').setLevel(logging.WARNING)

# Загрузка данных из Excel-файла
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/Прогноз SKU/SKU for prophet.xlsx')

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True)

# Подготовка данных для применения модели NeuralProphet
data = data.rename(columns={'Дата': 'ds', 'Код товара': 'SKU', 'Продажи, кг': 'y'})
data['y'] = data['y'].astype(float)

# Группировка данных по товару
grouped_data = data.groupby('SKU')

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Прогнозирование продаж для каждого товара
for sku, sku_data in grouped_data:
    if sku_data['y'].notnull().sum() < 2:
        continue  # Пропуск товаров с недостаточными данными

    # Обучение модели NeuralProphet
    model =  NeuralProphet(
            normalize="off",
            daily_seasonality=True,
            trainer_config={"default_root_dir": "/tmp"},
        )
    model.fit(sku_data[['ds', 'y']], freq='D')

    # Создание прогноза на 460 дней
    future = model.make_future_dataframe(sku_data, periods=460)
    forecast = model.predict(future)
    forecast['SKU'] = sku

    # Замена отрицательных значений прогноза на 0
    forecast['yhat1'] = forecast['yhat1'].where(forecast['yhat1'] > 0, 0)

    # Добавление прогнозов к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast])

# Сохранение прогнозов в Excel
selected_columns = ['ds', 'SKU', 'yhat1']
all_forecasts[selected_columns].to_excel('прогноз_продаж_по_SKU_neuralprophet.xlsx', index=False)
